### Newton's Optimization Method

**Analytical Solutions**. Analytical Solution to the Second Order Derivative (Elements of the Hessian Matrix)

![title](figures/Analytical_Hessian.png)



Creating a Logistic Regression Class based on Newtons Method.

![title](figures/LinearSystem.png)

Using the derivations from Above, we can now implement the logistic regression model with Newtons Method.

In [1]:
#Importing Needed Libaries
import numpy as np
np.random.seed(50)

#Logistic Regression Class
class LogisticRegression:

    def __init__(self, x, y):

        self.x = x #Input Random Data Matrix
        self.y = y #Input Random Data Target Vector
        self.beta_vector = self.initialize_betas() #Randomly Initializing Betas
        self.gradient_vector = self.gradient() #Analytically Solving for the Gradient Vector As an Extra Option
        self.hessian_matrix = self.hessian() #Analytically Solving for the Hessian Vector as an Extra Option
        self.optimized_betas = self.netwons_method() #Fit Method to optimize betas (note: not like SKlearn)
        self.predictions = self.predict() #Making Predictions with the optimized betas
        self.accuracy = self.get_accuracy() #Getting model accuracy
    
    def initialize_betas(self):

        beta_vector = np.random.random(len(self.x[0])) #Randomly Initializing The Beta Vector
        return beta_vector

    #Analytically Solving for the Gradient
    def gradient(self):

        gradient_vector = []
        gradient_value = 0

        for j in range(len(self.beta_vector)): #Finding First order partials for each beta

            for k in range(len(self.y)): #Summing through each data instance

                #First Order equation from above
                term_one = self.x[k][j] * y[k]
                term_two_numerator = -1 * (self.x[k][j] * np.exp(np.dot(self.x[k].T, self.beta_vector)))
                term_two_denominator = 1 + np.exp(np.dot(self.x[k].T, self.beta_vector))

                gradient_value += term_one + (term_two_numerator/term_two_denominator)

            gradient_vector.append(gradient_value)
            gradient_value = 0
        
        return np.array(gradient_vector) #Returning the gradient vector

    #Analytically Solving for the Hessian
    def hessian(self):

        hessian_matrix = []
        hessian_value = 0

        for j in range(len(self.beta_vector)): #Iterating through every beta for one pass

            for i in range(len(self.beta_vector)): #Iterating through every other beta for a second pass to build the Hessian Matrix

                for k in range(len(self.y)): #Iterating over every data instance 

                    #Hessian Second Order partial computations from above
                    numerator_term_one = 1 + np.exp(np.dot(self.x[k].T, self.beta_vector))
                    numerator_term_two = self.x[k][j] * self.x[k][i] * np.exp(np.dot(self.x[k].T, self.beta_vector))
                    numerator_term_three = self.x[k][j] * np.exp(np.dot(self.x[k].T, self.beta_vector))
                    numerator_term_four = self.x[k][i] * np.exp(np.dot(self.x[k].T, self.beta_vector))
                    numerator = (numerator_term_one * numerator_term_two) - (numerator_term_three * numerator_term_four)
                    
                    denominator = np.square(1 + np.exp(np.dot(self.x[k].T, self.beta_vector)))

                    hessian_value += numerator/denominator
                
                hessian_matrix.append(-1 * hessian_value)
                hessian_value = 0
        
        hessian_matrix = np.array(hessian_matrix)
        hessian_matrix = hessian_matrix.reshape([len(self.x[0]), len(self.x[0])]) #Reshaping the array to matrix form to finalize the hessian
        return hessian_matrix


    #Find difference of betas for convergence
    def abs_diff_betas(self, beta_step, current_beta):
        abs_diff_average = 0

        for i in range(len(beta_step)):
            abs_diff_average += np.abs(beta_step[i] - current_beta[i])
        
        return abs_diff_average / len(self.y)


    #Iterative Gradient Solver --> For newtons method
    def iterative_gradient_solver(self, beta_vect, index):

        first_order_beta = 0

        #Finding a first order partial from the loglikelihood function
        for k in range(len(self.y)):

            term_one = self.x[k][index] * y[k]
            term_two_numerator = -1 * (self.x[k][index] * np.exp(np.dot(self.x[k].T, beta_vect)))
            term_two_denominator = 1 + np.exp(np.dot(self.x[k].T, beta_vect))

            first_order_beta += (term_one + (term_two_numerator/term_two_denominator)) 
    
        return first_order_beta #Returns the first order partial (element in a gradient)

    #Iterative Hessian Solver
    def iterative_hessian_solver(self, beta_vect, index):

        second_order_beta = 0

        #Finding second order partial for a given beta (index is used to retrieve the beta)
        for k in range(len(self.y)):

            numerator_term_one = 1 + np.exp(np.dot(self.x[k].T, self.beta_vector))
            numerator_term_two = self.x[k][index] * self.x[k][index] * np.exp(np.dot(self.x[k].T, self.beta_vector))
            numerator_term_three = self.x[k][index] * np.exp(np.dot(self.x[k].T, self.beta_vector))
            numerator_term_four = self.x[k][index] * np.exp(np.dot(self.x[k].T, self.beta_vector))
            numerator = (numerator_term_one * numerator_term_two) - (numerator_term_three * numerator_term_four)
            denominator = np.square(1 + np.exp(np.dot(self.x[k].T, beta_vect)))

            second_order_beta += numerator/denominator
        
        return -1 * second_order_beta #Returns the second order partial (element in a hessian)

    
    #Newtons Method Functions

    def newtons_method_computation(self, index, beta_vect):

        #Computing newtons method as stated above
        beta_step = beta_vect[index] - (self.iterative_gradient_solver(beta_vect, index)/self.iterative_hessian_solver(beta_vect, index))
        return beta_step
    

    def newton_convergence(self, beta_step, current_beta):

        #Checking for convergence (betas do not change very much)
        flag = True
        for i in range(len(beta_step)):

            if(np.abs(beta_step[i] - current_beta[i]) > 1e-6):
                flag = False
                break
        
        return flag
    

    #Fit method to optimize betas
    def netwons_method(self):

        beta_steps = [] #Updated Beta vector
        beta_current = self.beta_vector

        num_iter = 0
        converged = False

        print("_Newtons Method Optimization_\n")
        print("Optimizing...")
        while(converged != True and num_iter < 30000): #Optimization

            for i in range(len(self.beta_vector)):

                beta_steps.append(self.newtons_method_computation(i,beta_current)) #Update Rule
            
            converged = self.newton_convergence(beta_steps, beta_current) #Check convergence
            num_iter += 1
            beta_current = beta_steps
            beta_steps = []
    
        if(converged):
            print("======================================================")
            print("Iteration:", num_iter)
            print("Average Absolute Beta-Step Difference:", self.abs_diff_betas(beta_steps, beta_current))
            print("Converge: True")
            print("\n_Optimization Terminated Successfully_")
            print("_Optimized Betas_", beta_current)
        
        else:
            print("_Failed to Converge_")
        return beta_current

    def predict(self):

        predictions = []
        predictions_vector = np.dot(self.optimized_betas, self.x.T) #Using optimized betas to predict

        for i in range(len(predictions_vector)):
            if(predictions_vector[i] >= 0.5): #Setting the classification constraint
                predictions.append(1)
            else:
                predictions.append(0)
        
        return predictions #Returns a list of predictions for the input data (in this case just the training data)

    def get_accuracy(self): #Gets accuracy of the predictions

        correct_classifications = 0
        for i in range(len(self.predictions)):
            if(self.predictions[i] == self.y[i]):
                correct_classifications+=1
        
        return 100 * correct_classifications/len(self.y) #Returns the accuracy percentage


In [2]:
#TESTING
x = np.random.random(size = (10, 5))
y = np.random.randint(0,2,10)
log_reg = LogisticRegression(x,y)
print("\n Accuracy:", log_reg.accuracy, "%")

_Newtons Method Optimization_

Optimizing...
Iteration: 1031
Average Absolute Beta-Step Difference: 0.0
Converge: True

_Optimization Terminated Successfully_
_Optimized Betas_ [-1.1728053530724705, 5.958321865116244, -0.7608323589840151, -2.383416137878991, -2.353692168975133]

 Accuracy: 70.0 %


**Statsmodels** Implementation and Results.

In [3]:
# Compute logistic regression coefficients# using Newton’s method
import statsmodels.api as sm
model = sm.Logit(y, x)
result = model.fit(method = "newton")
print(result.params)

Optimization terminated successfully.
         Current function value: 0.520427
         Iterations 6
[-1.17267262  5.95834742 -0.76087411 -2.38353518 -2.35370509]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=be83efb1-3870-4fac-886c-ad7be4823257' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>